In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import clang.cindex
from parser.parser import *

In [3]:
fn_cpp = "source_cpp/example1.cpp"

In [5]:
# 1. Operator 정보 전체 출력으로 수정
# 2. Node가 제대로 바뀐게 맞나? get_Children으로 확인. 

tu = parse_code_snippet(fn_cpp, loc=True)

Found ciphertext reference
  CursorKind.UNEXPOSED_EXPR CipherNode
CursorKind.TRANSLATION_UNIT source_cpp/example1.cpp [<SourceLocation file None, line 0, column 0>]
Found ciphertext reference
Found ciphertext reference
    CursorKind.UNEXPOSED_EXPR CipherNode
  CursorKind.FUNCTION_DECL fun : FunctionProto [<SourceLocation file 'source_cpp/example1.cpp', line 1, column 12>]
Found ciphertext reference
      CursorKind.UNEXPOSED_EXPR CipherNode
    CursorKind.PARM_DECL a : Int [<SourceLocation file 'source_cpp/example1.cpp', line 1, column 27>]
    CursorKind.COMPOUND_STMT  [<SourceLocation file 'source_cpp/example1.cpp', line 1, column 30>]
      CursorKind.DECL_STMT  [<SourceLocation file 'source_cpp/example1.cpp', line 3, column 5>]
        CursorKind.VAR_DECL b : Auto [<SourceLocation file 'source_cpp/example1.cpp', line 3, column 10>]
          CursorKind.BINARY_OPERATOR  : Dependent (Operator: -a+2) [<SourceLocation file 'source_cpp/example1.cpp', line 3, column 14>]
            Cur

In [17]:
tu = parse_code_snippet(fn_cpp, loc=False, ignore_main=True)

CursorKind.TRANSLATION_UNIT source_cpp/example1.cpp
  CursorKind.FUNCTION_DECL fun : FunctionProto
    CursorKind.PARM_DECL a : Int
    CursorKind.COMPOUND_STMT 
      CursorKind.DECL_STMT 
        CursorKind.VAR_DECL b : Auto
          CursorKind.BINARY_OPERATOR  : Dependent (Operator: -)
            CursorKind.UNARY_OPERATOR  : Dependent (Operator: -)
              CursorKind.UNEXPOSED_EXPR  : Dependent
            CursorKind.INTEGER_LITERAL  : Int
      CursorKind.DECL_STMT 
        CursorKind.VAR_DECL c : Auto
          CursorKind.BINARY_OPERATOR  : Dependent (Operator: a)
            CursorKind.UNEXPOSED_EXPR  : Dependent
            CursorKind.UNEXPOSED_EXPR  : Dependent
      CursorKind.RETURN_STMT 
        CursorKind.UNEXPOSED_EXPR  : Dependent


1. PlainNode와 CipherNode를 판별
- CipherNode이거나, CipherNode와 binary operator에 참여하는 경우
- if child.kind == clang.cindex.CursorKind.BINARY_OPERATOR:
- - for _ in child.get_tokens():
  -     if _.spelling == "a":
  -         return True

    여기에다가 CipherNode를 추가하나? 아님 CipherNode로 바꾸나?
    Desilo AST에서 CipherNode의 역할은 무엇인가?

- 이 둘에게는 Params{} 특성 부여
- PlainNode는 params가 모두 null
- - "params": {
			  "scale": null,
			  "min_scale": null,
			  "level": null
			},
- - CipherNode는 params 값을 어디서 받아오지...?    -> Main function에서 설정하는 단계가 있고, 거기서 받아오는 메커니즘 필요함.

## Meeting
`a = program.add_secret("a", 40)` 여기서 scale 정보 추가해줌. Level 정보는 기본이 1000인데 의미 없음. (사용하지 않음) 
isexecuted: 사용하지 않음 
JSON에서 어떤 변수가 Ciphertext인지, Plaintext인지도 굳이 따지지 않음. 사실상 토큰을 JSON으로 변환하는 것에 가까움. 

demo2  
for i in range(5):  
에서, 루프 푸는 것도 nxGraph툴이 하는 것 같음. 
5가 아니라 변수가 들어가는 상황은 지원하지 않음.   

if 문도 지원하지 않음.  



2. Unary Operation인 negation 먼저 처리
- Negation 판별
- CursorKind.UNARY_OPERATOR  : Dependent (Operator: -a) 에서..
- - `if clang.cindex.CursorKind.UNARY_OPERATOR and operator == "-"`
 
3. Binary Operator 처리
- Addition : `if clang.cindex.CursorKind.BINARY_OPERATOR and operator == "+"`
- Multiplication : `if clang.cindex.CursorKind.BINARY_OPERATOR and operator == "*"`
- Subtraction : `if clang.cindex.CursorKind.BINARY_OPERATOR and operator == "-"`

CursorKind.DECL_STMT  (
  CursorKind.VAR_DECL b : Auto

-> b = 

In [9]:
for ch in tu.cursor.get_children():
    print(ch)

In [ ]:
# get_children -> list_iterator

In [11]:
ch2 = ch.get_children()

In [12]:
ch2